In [6]:
import os
import numpy as np
import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_raw.fif')
sample_data_proj_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_eog_proj.fif')

%matplotlib widget

Some artifacts are restricted to certain frequencies and can therefore be fixed by filtering. An artifact that typically affects only some frequencies is due to the power line.

Power-line noise is a noise created by the electrical network. It is composed of sharp peaks at 50Hz (or 60Hz depending on your geographical location). Some peaks may also be present at the harmonic frequencies, i.e. the integer multiples of the power-line frequency, e.g. 100Hz, 150Hz, … (or 120Hz, 180Hz, …).




In [7]:
tmin, tmax = 0, 20 # use the first 20 seconds of the data
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(tmin, tmax).load_data()

Opening raw data file /Users/markyousef/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Current compensation grade : 0
Reading 0 ... 12012  =      0.000 ...    20.000 secs...


<Raw  |  sample_audvis_raw.fif, n_channels x n_times : 376 x 12013 (20.0 sec), ~38.1 MB, data loaded>

In [8]:
raw.info['bads'] = ['MEG 2443', 'EEG 053'] # bads + 2 more

In [9]:
fmin, fmax = 2, 300 # look at the frequencies between 2 and 300Hz
n_fft = 2048 # the FFT size – ideally a power of 2

In [10]:
selection = mne.read_selection('Left-temporal') # select a subset of channels
picks = mne.pick_types(raw.info, meg='mag', eeg=False, eog=False, stim=False, exclude='bads', selection=selection)

raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 3.410 (s)


## Removing power-line noise with a notch filter

In [12]:
freqs = np.arange(60, 241, 60); freqs

array([ 60, 120, 180, 240])

In [15]:
raw.notch_filter(freqs, picks, fir_design="firwin")
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3965 samples (6.602 sec)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 3.410 (s)


## Removing power-line noise with low-pass filtering

In [16]:
raw.filter(None, 50., fir_design='firwin') # low pass filter below 50Hz
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 159 samples (0.265 sec)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 3.410 (s)


## High-pass filtering to remove slow drifts

In [18]:
raw.filter(1.0, None, fir_design='firwin')
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1983 samples (3.302 sec)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 3.410 (s)


In [19]:
raw.filter(1, 50., fir_design='firwin') # band-pass filtering in the range 1 Hz - 50 Hz
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1983 samples (3.302 sec)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 3.410 (s)


## Downsampling and decimation

When performing experiments where timing is critical, a signal with a high sampling rate is desired. However, having a signal with a much higher sampling rate than necessary needlessly consumes memory and slows down computations operating on the data. To avoid that, you can downsample your time series. Since downsampling raw data reduces the timing precision of events, it is recommended only for use in procedures that do not require optimal precision, e.g. computing EOG or ECG projectors on long recordings.

In [20]:
raw.resample(100, npad='auto') # set sampling frequency to 100Hz
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=True);

25 events found
Event IDs: [ 1  2  3  4  5 32]
25 events found
Event IDs: [ 1  2  3  4  5 32]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Effective window size : 10.010 (s)


To avoid this reduction in precision, the suggested pipeline for processing final data to be analyzed is:

* low-pass the data with `mne.io.Raw.filter()`

* Extract epochs with `mne.Epochs`

* Decimate the Epochs object using `mne.Epochs.decimate()` or the decim argument to the `mne.Epochs` object.